In [1]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch


# model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")

In [9]:
import torch
x = torch.randn(100, 100, 128)
y = torch.randn(100, 100, 128)

torch.nn.functional.cosine_similarity(x, y, dim=-1).shape

torch.Size([100, 100])

In [2]:
tokenizer("""Prompt:
Write a short story about Antonio.

Model output:

""")

{'input_ids': [2, 70517, 236787, 107, 6974, 496, 2822, 3925, 1003, 22982, 236761, 108, 4968, 3938, 236787, 108], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

input_text = "Einstein was"
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]
print(input_ids)

# 1️⃣ Generate new tokens
outputs = model.generate(
    input_ids,
    max_new_tokens=10,
    do_sample=False,  # deterministic
    return_dict_in_generate=True,
    output_scores=True
)

generated_ids = outputs.sequences  # input + generated
print("Generated text:", tokenizer.decode(generated_ids[0]))

# 2️⃣ Get logits for the entire sequence
with torch.no_grad():
    # Run the model on the full sequence (input + generated)
    logits = model(generated_ids).logits  # shape: [1, seq_len, vocab_size]

print("Logits shape:", logits.shape)

In [ ]:
outputs

In [ ]:
model(input_ids)

In [ ]:
print(tokenizer.decode(outp[0], skip_special_tokens=True))

In [2]:
from hfdebugger import HFModelDebugger, debug_huggingface_model

debugger, outputs, input_ids = debug_huggingface_model(
    model, tokenizer, 
    input_text="Einstein was born",
    save_debug_data=True
)

Registering hooks on ALL modules... (filtering by name in hook)
Registered 314 hooks (filtering applied in hook function)
Input text: 'Einstein was born'
Tokenized: ['<bos>', 'Einstein', '▁was', '▁born']
Input IDs shape: torch.Size([1, 4])
Input IDs: tensor([[     2, 147505,    691,   8132]])


`sdpa` attention does not support `output_attentions=True` or `head_mask`. Please set your attention to `eager` if you want any of these features.



000_model.embed_tokens: Gemma3TextScaledWordEmbedding (model.embed_tokens)

  → INPUT[0]: model.embed_tokens_input_0
     Shape: [1, 4]
     Dtype: torch.int64
     Mean: 39082.500000, Std: 72375.328125
     First 10: [2, 147505, 691, 8132]
     Last 10:  [2, 147505, 691, 8132]
     Zeros: 0, Total: 4

  → OUTPUT[0]: model.embed_tokens_output
     Shape: [1, 4, 640]
     Dtype: torch.float32
     Mean: -0.010992, Std: 1.031457
     First 10: [-0.36131492257118225, 0.27021417021751404, 0.3381537199020386, 0.3149925172328949, 0.12352646887302399, -0.07913414388895035, 0.10345342010259628, 0.09496097266674042, 0.04844553768634796, -0.007479141931980848]
     Last 10:  [0.8461563587188721, 1.1982067823410034, -0.021424122154712677, -1.1426198482513428, -1.865249752998352, -1.5379046201705933, 0.021424122154712677, 0.47557690739631653, 0.4354308247566223, 0.25940558314323425]
     Zeros: 0, Total: 2560

  → PARAMETER: weight
     Shape: [262144, 640]
     Mean: -0.000165
     First 10: [-0

In [ ]:
outputs.logits

In [ ]:
outputs.logits.shape

In [ ]:
print(model._modules["model"].layers[0]._modules["self_attn"])

In [ ]:
vars(model._modules["model"].rotary_emb)



In [ ]:
vars(model._modules["model"].rotary_emb_local)

In [ ]:
import torch
# Step 1: Clear all existing hooks
layers = model._modules["model"].layers
for layer in layers:
    if hasattr(layer, 'input_layernorm'):
        layer.input_layernorm._forward_hooks.clear()
    if hasattr(layer, 'self_attn'):
        layer.self_attn._forward_hooks.clear()
    if hasattr(layer, 'post_attention_layernorm'):
        layer.post_attention_layernorm._forward_hooks.clear()

# Step 2: Clear activations dictionary
activations = {}

# Step 3: Define the NEW hook function that handles tuples
def get_hook(name):
    def hook(module, input, output):
        print(f"Hook {name} fired! Type of output: {type(output)}")
        
        # Handle different output types
        if isinstance(output, tuple):
            # For attention modules that return (hidden_states, attention_weights)
            # Store the main output (hidden states)
            activations[name] = output[0].detach()
            print(f"  Tuple length: {len(output)}")
            print(f"  Main output shape: {output[0].shape}")
            if len(output) > 1 and output[1] is not None:
                print(f"  Attention weights shape: {output[1].shape}")
        else:
            # For layer norm and other modules that return single tensors
            activations[name] = output.detach()
            print(f"  Tensor shape: {output.shape}")
    
    return hook

# Step 4: Register hooks with the NEW function
layers[0].input_layernorm.register_forward_hook(get_hook("layer_0_input_ln"))
layers[0].self_attn.register_forward_hook(get_hook("layer_0_self_attn"))
layers[0].post_attention_layernorm.register_forward_hook(get_hook("layer_0_post_self_attn_ln"))


In [ ]:

# Prepare input
text = "Hi Gemma!"
inputs = tokenizer(text, return_tensors="pt")
print(inputs)

# Forward pass with hidden states
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)

# Extract hidden states
hidden_states = outputs.hidden_states  # Tuple of all layer outputs
print(f"Number of layers: {len(hidden_states)}")
print(f"Shape of each hidden state: {hidden_states[0].shape}")  # [batch, seq_len, hidden_dim]

# Look at specific layers
print(f"Input embeddings: {hidden_states[0][0, 0, :5]}")  # Last token, first 5 dims
print(f"After layer 0: {hidden_states[1][0, 0, :5]}")
print(f"After layer 1: {hidden_states[2][0, 0, :5]}")
print(f"Final layer output: {hidden_states[-1][0, 0, :5]}")


for name, value in activations.items():
    print(f"{name}: {value}")

In [ ]:
model._modules["model"]
lm_head = model._modules["lm_head"].weight.flatten().tolist()
with open("gemma-3-270M/lm_head.txt", "w") as f:
    for w in lm_head:
        f.write(f"{w}\n")

In [ ]:
vars(model)

In [ ]:
id2token = {id_: tok for tok, id_ in tok.get_vocab().items()}
id2token = 
print({k: id2token[k] for k in list(id2token)[:10]})

In [ ]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("google/gemma-3-270m")
print(len(tok))

In [ ]:

token2id = tok.get_vocab() 

# sort by value
token2id = dict(sorted(token2id.items(), key=lambda item: item[1]))
import json
with open("GemmaTokenizerFast.json", "w") as f:
    json.dump(token2id, f, indent=4)

In [ ]:
from flashml.inspect import inspect_tokenizer

inspect_tokenizer(tok)

In [ ]:

x = "Hello there martin, here's your special delight zone!"

tok.encode(x)